In [2]:
pip install requests pymongo pandas

  Using cached pymongo-4.15.0-cp313-cp313-win_amd64.whl.metadata (22 kB)
  Using cached dnspython-2.8.0-py3-none-any.whl.metadata (5.7 kB)
Using cached pymongo-4.15.0-cp313-cp313-win_amd64.whl (961 kB)
Using cached dnspython-2.8.0-py3-none-any.whl (331 kB)

   ---------------------------------------- 0/2 [dnspython]
   -------------------- ------------------- 1/2 [pymongo]
   -------------------- ------------------- 1/2 [pymongo]
   -------------------- ------------------- 1/2 [pymongo]
   ---------------------------------------- 2/2 [pymongo]

Note: you may need to restart the kernel to use updated packages.


In [3]:
import os
from datetime import datetime, timezone
import requests
from pymongo import MongoClient, ASCENDING
from datetime import datetime, timezone

In [24]:
API_KEY = "cb3b7ff5870bf1836fcc688dc62dcb4f"
CITY = "Chengdu"

In [6]:
MONGO_URL = os.getenv("MONGO_URL","mongodb://127.0.0.1:27017")
DB_NAME = "sales_weatherinfo_db"
WEATHER_COL = "weather"

In [7]:
def midnight_utc_naive(dt_utc: datetime) -> datetime:
    dt_utc = dt_utc.astimezone(timezone.utc)
    return datetime(dt_utc.year,dt_utc.month,dt_utc.day,0,0,0,0)

In [9]:
def ensure_indexes(col):
    col.create_index([("weather_date",ASCENDING),("city",ASCENDING)], unique=True)
    col.create_index([("city",ASCENDING),("weather_date",ASCENDING)])

In [17]:
def upsert_weather(doc: dict, col):
    col.update_one(
        {"weather_date": doc["weather_date"], "city": doc["city"]},
        {"$set": doc},
        upsert = True
    )

In [18]:
def fetch_weather_live(api_key: str, city:str) -> dict:
    url = "https://api.openweathermap.org/data/2.5/weather"
    params = {"q":city, "appid":api_key.strip(), "units":"metric"}

    try:
        r = requests.get(url, params=params, timeout=20)
    except requests.RequestException as e:
        raise RuntimeError(f"Network error calling OpenWeather: {e}")

    if not r.ok:
        try:
            msg = r.json()
        except Exception:
            msg = r.text
        raise RuntimeError(f"OpenWeather error {r.status_code}: {msg} | url={r.url}")

    data = r.json()
    return{
        "weather_date":datetime.now(timezone.utc).replace(hour=0,minute=0,second=0,microsecond=0),
        "city": city,
        "temp_c": float(data["main"]["temp"]),
        "humidity": int(data["main"]["humidity"]),
        "description": str(data["weather"][0]["description"]),
        "fetched_at": datetime.now(timezone.utc),
        "source": "openweather"
    }

In [25]:
def main():
    if not API_KEY or len(API_KEY.strip()) != 32:
        raise ValueError("OpenWeather API key looks invalid. Except 32 chars.")

    client = MongoClient(MONGO_URL)
    col = client[DB_NAME][WEATHER_COL]

    ensure_indexes(col)
    doc = fetch_weather_live(API_KEY, CITY)
    upsert_weather(doc, col)

    print(f"Stored live weather for {doc['city']} on {doc['weather_date'].date()}.")

In [26]:
if __name__ == "__main__":
    main()

Stored live weather for Chengdu on 2025-09-13.
